In [1]:
import scipy.sparse as sps
import numpy as np
import pandas as pd
import scipy as sc
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = np.genfromtxt('../input/train.csv' , delimiter=',',dtype=int)[1:]
test = np.genfromtxt('../input/target_playlists.csv', delimiter = ',', dtype=int)[1:]

playlist = data[:,0]
song = data[:,1]

unique_playlist = list(set(playlist))
unique_songs = list(set(song))

num_playlists = len(unique_playlist)
num_songs = len(unique_songs)

mat = sps.lil_matrix((num_playlists,num_songs))
mat[playlist,song] = 1
URM_csr = sc.sparse.csr_matrix(mat)


In [3]:
similarities = cosine_similarity(URM_csr.transpose(), dense_output=False)

In [4]:
def predict(URM_csr, similarities, playlist):
    user = np.array(URM_csr.getrow(playlist).todense())[0]
    predicted_ratings = similarities.transpose().dot(user)
    
    return predicted_ratings

In [5]:
class ItemBasedCollaborativeFilteringRecommender(object):
    
    def fit(self, URM_csr):
        self.similarities = cosine_similarity(URM_csr, dense_output=False)
        self.URM_csr = URM_csr
    
    def recommend(self, user_id, at=10, remove_seen=True):
        user = np.array(self.URM_csr.getrow(playlist).todense())[0]
        itemPopularity = self.similarities.transpose().dot(user)
        self.popularItems = np.argsort(itemPopularity)
        self.popularItems = np.flip(self.popularItems, axis = 0)
        
        if remove_seen:
            unseen_items_mask = np.in1d(self.popularItems, self.URM_csr[user_id].indices,
                                        assume_unique=True, invert = True)

            unseen_items = self.popularItems[unseen_items_mask]
            
            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popularItems[0:at]
            
        #recommended_items = " ".join(str(i) for i in recommended_items)
        return recommended_items